In [1]:
import pandas as pd
import os
import numpy as np
import h5py
import pickle
import matplotlib.pyplot as plt
import importlib
import functions
import spectrogram_plotting_functions
import scipy.stats
importlib.reload(functions)
importlib.reload(spectrogram_plotting_functions)


<module 'spectrogram_plotting_functions' from 'c:\\Users\\sinha\\cpl_analysis_naman\\spectrogram_plotting_functions.py'>

In [2]:
import getpass
user= (getpass.getuser())
print("Hello", user)

import glob
base='C:\\Users\\{}\\Dropbox\\CPLab'.format(user)
files = glob.glob(base+'\\all_data_mat_filtered\\*.mat')
savepath = base+'\\results\\'
print(files)

Hello sinha
['C:\\Users\\sinha\\Dropbox\\CPLab\\all_data_mat_filtered\\20230529_dk1_nocontext.mat', 'C:\\Users\\sinha\\Dropbox\\CPLab\\all_data_mat_filtered\\20230529_dk3_nocontext.mat', 'C:\\Users\\sinha\\Dropbox\\CPLab\\all_data_mat_filtered\\20230529_dk5_nocontext.mat', 'C:\\Users\\sinha\\Dropbox\\CPLab\\all_data_mat_filtered\\20230529_dk6_nocontext.mat', 'C:\\Users\\sinha\\Dropbox\\CPLab\\all_data_mat_filtered\\20230531_dk1_nocontext_day2.mat', 'C:\\Users\\sinha\\Dropbox\\CPLab\\all_data_mat_filtered\\20230531_dk3_nocontext_day2.mat', 'C:\\Users\\sinha\\Dropbox\\CPLab\\all_data_mat_filtered\\20230531_dk5_nocontext_day2.mat', 'C:\\Users\\sinha\\Dropbox\\CPLab\\all_data_mat_filtered\\20230531_dk6_nocontext_day2.mat', 'C:\\Users\\sinha\\Dropbox\\CPLab\\all_data_mat_filtered\\20230609_dk1_BW_nocontext_day1.mat', 'C:\\Users\\sinha\\Dropbox\\CPLab\\all_data_mat_filtered\\20230609_dk3_BW_nocontext_day1.mat', 'C:\\Users\\sinha\\Dropbox\\CPLab\\all_data_mat_filtered\\20230610_dk1_BW_noconte

In [ ]:

keyboard_dict={'98':'b','119':'w','120':'nc','49':'1','48':'0'}
#files=[f'C:\\Users\\{user}\\Dropbox\\CPLab\\all_data_mat_filtered\\20230615_dk6_BW_context_day1.mat'] #This is just for testing purposes
events_codes_all = {}
compiled_data_all_epochs = pd.DataFrame()

for file in files:
    # Initialize an empty DataFrame to store the results for all epochs
    ## Get the date, mouse_id and task from the file name
    base_name = os.path.basename(file)
    base_name, _ = os.path.splitext(base_name)

    date, mouse_id, task=functions.exp_params(base_name)
    print(date, mouse_id, task)
    ## Open the file
    f=h5py.File(file, 'r')
    channels = list(f.keys())
    print(channels)

    if 'Keyboard' in channels:
        events = f['Keyboard']
    elif 'keyboard' in channels:
        events = f['keyboard']
    elif 'memory' in channels:
        events = f['memory']
    elif 'Memory' in channels:
        events = f['Memory']

    events_codes=np.array(events['codes'][0])
    events_times=np.array(events['times'][0])
    events_codes_all[base_name] = events_codes
    #Generating epochs from events (epochs are basically start of a trial and end of a trial)
    epochs=functions.generate_epochs_with_first_event(events_codes, events_times)

    # Experiment Start time
    first_event=events_times[0]

    for channeli in channels:
        if "AON" in channeli or  "vHp" in channeli :
            
            channel_id=channeli
            # Extracting raw data and time
            data_all=f[channeli]
            raw_data=np.array(data_all['values']).flatten()
            raw_time = np.array(data_all['times']).flatten()
            sampling_rate = int(1/data_all['interval'][0][0])
            print(raw_data.shape, raw_time.shape, sampling_rate)
            # Normalizing the data by subtracting the mean and std of data 30sec before the first event
            normalized_data,time,data_before=functions.data_normalization(raw_data,raw_time,first_event, sampling_rate)

            # Applying a notch filter
            notch_filtered_data=functions.iir_notch(normalized_data, sampling_rate,60)
            total=notch_filtered_data
            # Extracting the bands
            beta=functions.beta_band(notch_filtered_data, sampling_rate)
            gamma=functions.gamma_band(notch_filtered_data, sampling_rate)
            theta=functions.theta_band(notch_filtered_data, sampling_rate)

            all_bands=[total,beta, gamma, theta]

            for i,epochi in enumerate(epochs):
                compiled_data = pd.DataFrame()

                door_timestamp = epochi[0][0]
                trial_type = epochi[0][1]
                dig_type = epochi[1, 1]
                print(dig_type)
                dig_timestamp = epochi[1, 0]
                print(door_timestamp,trial_type,dig_timestamp,dig_type)
                for bandi in all_bands:
                    data_complete_trial=functions.extract_complete_trial_data(bandi,time,door_timestamp,dig_timestamp,sampling_rate)
                        # Create a DataFrame for the current bandi
                    bandi_data = pd.DataFrame({
                        'data_complete_trial': [data_complete_trial]

                    })
                    
                    # Concatenate the current bandi DataFrame with the compiled_data DataFrame along axis=1
                    compiled_data = pd.concat([compiled_data, bandi_data], axis=1)
                compiled_data.columns = ['total_complete_trial',
                            'beta_complete_trial',
                            'gamma_complete_trial',
                            'theta_complete_trial']
                compiled_data.insert(0, 'rat', mouse_id)
                compiled_data.insert(1, 'date', date)
                compiled_data.insert(2, 'experiment', task)
                compiled_data.insert(3, 'channel', channel_id)
                compiled_data.insert(4, 'trial', i)
                compiled_data.insert(5, 'timestamps', [[door_timestamp, dig_timestamp]])

                compiled_data.insert(6, 'side', keyboard_dict[str(int(trial_type))])
                compiled_data.insert(7, 'correct?', keyboard_dict[str(int(dig_type))])
                compiled_data.insert(8, 'first 30 seconds power', functions.calculate_power_1D(data_before))
                compiled_data.insert(9, 'time', [time])
                
                compiled_data_all_epochs = pd.concat([compiled_data_all_epochs, compiled_data], axis=0, ignore_index=True)
    

In [3]:
import os
import h5py
import numpy as np
import pandas as pd
from concurrent.futures import ProcessPoolExecutor

keyboard_dict = {'98': 'b', '119': 'w', '120': 'nc', '49': '1', '48': '0'}
# files = [f'C:\\Users\\{user}\\Dropbox\\CPLab\\all_data_mat_filtered\\20230615_dk6_BW_context_day1.mat'] # This is just for testing purposes
events_codes_all = {}
compiled_data_all_epochs = []

def process_file(file):
    compiled_data_list = []
    
    # Get the date, mouse_id and task from the file name
    base_name = os.path.basename(file)
    base_name, _ = os.path.splitext(base_name)
    date, mouse_id, task = functions.exp_params(base_name)
    print(date, mouse_id, task)
    
    # Open the file
    with h5py.File(file, 'r') as f:
        channels = list(f.keys())
        print(channels)

        if 'Keyboard' in channels:
            events = f['Keyboard']
        elif 'keyboard' in channels:
            events = f['keyboard']
        elif 'memory' in channels:
            events = f['memory']
        elif 'Memory' in channels:
            events = f['Memory']

        events_codes = np.array(events['codes'][0])
        events_times = np.array(events['times'][0])
        events_codes_all[base_name] = events_codes
        
        # Generating epochs from events (epochs are basically start of a trial and end of a trial)
        epochs = functions.generate_epochs_with_first_event(events_codes, events_times)
        
        # Experiment Start time
        first_event = events_times[0]

        for channeli in channels:
            if "AON" in channeli or "vHp" in channeli:
                channel_id = channeli
                # Extracting raw data and time
                data_all = f[channeli]
                raw_data = np.array(data_all['values']).flatten()
                raw_time = np.array(data_all['times']).flatten()
                sampling_rate = int(1 / data_all['interval'][0][0])
                print(raw_data.shape, raw_time.shape, sampling_rate)
                
                # Normalizing the data by subtracting the mean and std of data 30sec before the first event
                normalized_data, time, data_before = functions.data_normalization(raw_data, raw_time, first_event, sampling_rate)
                
                # Applying a notch filter
                notch_filtered_data = functions.iir_notch(normalized_data, sampling_rate, 60)
                total = notch_filtered_data
                
                # Extracting the bands
                beta = functions.beta_band(notch_filtered_data, sampling_rate)
                gamma = functions.gamma_band(notch_filtered_data, sampling_rate)
                theta = functions.theta_band(notch_filtered_data, sampling_rate)
                
                all_bands = [total, beta, gamma, theta]

                for i, epochi in enumerate(epochs):
                    door_timestamp = epochi[0][0]
                    trial_type = epochi[0][1]
                    dig_type = epochi[1, 1]
                    dig_timestamp = epochi[1, 0]
                    print(dig_type)
                    print(door_timestamp, trial_type, dig_timestamp, dig_type)
                    
                    band_data_dict = {}
                    for bandi, band_name in zip(all_bands, ['total', 'beta', 'gamma', 'theta']):
                        data_complete_trial = functions.extract_complete_trial_data(bandi, time, door_timestamp, dig_timestamp, sampling_rate)
                        band_data_dict[f'{band_name}_complete_trial'] = [data_complete_trial]
                    
                    compiled_data = pd.DataFrame(band_data_dict)
                    compiled_data.insert(0, 'rat', mouse_id)
                    compiled_data.insert(1, 'date', date)
                    compiled_data.insert(2, 'experiment', task)
                    compiled_data.insert(3, 'channel', channel_id)
                    compiled_data.insert(4, 'trial', i)
                    compiled_data.insert(5, 'timestamps', [[door_timestamp, dig_timestamp]])
                    compiled_data.insert(6, 'side', keyboard_dict[str(int(trial_type))])
                    compiled_data.insert(7, 'correct?', keyboard_dict[str(int(dig_type))])
                    compiled_data.insert(8, 'first 30 seconds power', functions.calculate_power_1D(data_before))
                    compiled_data.insert(9, 'time', [time])

                    compiled_data_list.append(compiled_data)
    
    return compiled_data_list

In [8]:
import gc
gc.collect()
del compiled_data_all_epochs

In [5]:
# Process files sequentially
for file in files:
    compiled_data_list = process_file(file)
    compiled_data_all_epochs.extend(compiled_data_list)
# Flatten the list of lists

compiled_data_all_epochs = pd.concat(compiled_data_all_epochs, axis=0, ignore_index=True)

# Save the DataFrame with compression and chunking
filename = 'compiled_data_all_rats_all_epoch.h5'
compiled_data_all_epochs.to_hdf(os.path.join(savepath,filename), key='df', mode='w', complevel=9, complib='blosc')

# Free up memory
del compiled_data_all_epochs, compiled_data_list, events_codes_all
gc.collect()

20230529 dk1 nocontext
['Keyboard', 'LFP1_AON', 'LFP1_vHp', 'LFP2_AON', 'LFP2_vHp', 'LFP3_AON', 'LFP4_AON', 'Ref', 'Respirat', 'file']
(1516022,) (1516022,) 2000
normalizing data
notch filter applied
filtering beta band
filtering gamma band
filtering theta band
49.0
181.20385499999998 120.0 182.713855 49.0
48.0
237.771855 120.0 239.46185499999999 48.0
49.0
279.551855 120.0 280.29184999999995 49.0
48.0
348.953855 120.0 349.84385 48.0
49.0
466.78984999999994 120.0 468.25985499999996 49.0
49.0
526.9038549999999 120.0 528.223855 49.0
49.0
570.771855 120.0 571.4418549999999 49.0
48.0
617.237855 120.0 618.4178549999999 48.0
49.0
669.49786 120.0 670.737855 49.0
49.0
705.3358549999999 120.0 706.5758549999999 49.0
(1516022,) (1516022,) 2000
normalizing data
notch filter applied
filtering beta band
filtering gamma band
filtering theta band
49.0
181.20385499999998 120.0 182.713855 49.0
48.0
237.771855 120.0 239.46185499999999 48.0
49.0
279.551855 120.0 280.29184999999995 49.0
48.0
348.953855 120.

KeyboardInterrupt: 

In [ ]:
import sys
import pandas as pd

# Assuming compiled_data_all_epochs is a pandas DataFrame
in_memory_size = sys.getsizeof(compiled_data_all_epochs)
print(f"In-memory size: {in_memory_size / (1024 ** 3):.2f} GB")

# Rough estimate of compressed size (assuming a compression ratio of 2:1)
compression_ratio = 9
estimated_compressed_size = in_memory_size / compression_ratio
print(f"Estimated compressed size: {estimated_compressed_size / (1024 ** 3):.2f} GB")